## Entendimiento de los datos

Montar Google Drive y cargar archivos:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
from scipy.io import loadmat
import random

# Ruta a la carpeta "base" en Google Drive
ruta_a_tus_archivos = '/content/drive/My Drive/training/'

path = '/content/drive/My Drive/training/'
all_files = sorted(os.listdir(path))

hea_files = [f for f in all_files if f.endswith('.hea')]
mat_files = [f for f in all_files if f.endswith('.mat')]

# Asumimos que por cada archivo .hea hay un archivo .mat con el mismo nombre base
paired_files = [(os.path.join(path, h), os.path.join(path, m)) for h, m in zip(hea_files, mat_files)]

# Crear una lista para almacenar la información de los ECG
ecg_info = []




Mounted at /content/drive


Leer e imprimir información de registros seleccionados al azar:

In [ ]:
# Recorrer los archivos emparejados .hea y .mat
for hea_file, mat_file in paired_files:
    # Leer y procesar el archivo .hea
    with open(hea_file, 'r') as f:
        hea_content = f.read()

    # Leer el archivo .mat
    mat_content = loadmat(mat_file)

    ecg_info.append({"File": hea_file, "HEA Content": hea_content, "MAT Content": mat_content})

# Seleccionar al azar 5 registros y ordenarlos por nombre de archivo
random_sample = random.sample(ecg_info, min(5, len(ecg_info)))
random_sample_sorted = sorted(random_sample, key=lambda x: x["File"])

# Imprimir la información de los 5 registros seleccionados al azar
for record in random_sample_sorted:
    print("Información para el archivo:", record["File"])
    print("Contenido del archivo .hea:")
    print(record["HEA Content"])
    print("Contenido del archivo .mat:")
    print(record["MAT Content"])
    print("\n")


Información para el archivo: /content/drive/My Drive/training/a550s.hea
Contenido del archivo .hea:
a550s 4 250 75000
a550s.mat 16+24 407.3/mV 16 0 299 -26355 0 II
a550s.mat 16+24 553.5/mV 16 0 964 -9225 0 V
a550s.mat 16+24 5894/NU 16 0 11377 18788 0 PLETH
a550s.mat 16+24 8193/NU 16 0 1401 -26638 0 RESP
#Asystole
#False alarm

Contenido del archivo .mat:
{'val': array([[   299,    400,    404, ..., -32768, -32768, -32768],
       [   964,   1306,   1350, ..., -32768, -32768, -32768],
       [ 11377,  15453,  15941, ...,  15962,  16084,  16253],
       [  1401,   1902,   1962, ..., -32768, -32768, -32768]], dtype=int16)}


Información para el archivo: /content/drive/My Drive/training/b265l.hea
Contenido del archivo .hea:
b265l 3 250 82500
b265l.mat 16+24 1.365e+04/mV 16 0 3547 21780 0 II
b265l.mat 16+24 1.004e+04/mV 16 0 8546 7882 0 V
b265l.mat 16+24 8498/NU 16 0 2566 3073 0 PLETH
#Bradycardia
#True alarm

Contenido del archivo .mat:
{'val': array([[ 3547,  3987,  3440, ..., 12753, 1268

Obtener comentarios únicos de archivos .hea:

In [ ]:
# Crear una lista para almacenar los comentarios
comments_list = []

# Recorrer los archivos .hea
for hea_file in hea_files:
    hea_path = os.path.join(path, hea_file)
    with open(hea_path, 'r') as f:
        hea_content = f.read()
        # Buscar líneas que comiencen con '#'
        comments = [line.strip() for line in hea_content.splitlines() if line.startswith('#')]
        # Agregar los comentarios a la lista
        comments_list.extend(comments)

# Obtener comentarios únicos
unique_comments = list(set(comments_list))

# Imprimir la lista de comentarios únicos
print("Lista de Comentarios Posibles:")
for comment in unique_comments:
    print(comment)


Lista de Comentarios Posibles:
#Asystole
#Ventricular_Flutter_Fib
#Tachycardia
#Bradycardia
#True alarm
#Ventricular_Tachycardia
#False alarm


Crear DataFrame con información de los ECG:

In [ ]:
import pandas as pd

# Crear una lista para almacenar la información de los ECG
ecg_info = []

# Recorrer los archivos emparejados .hea y .mat
for hea_file, mat_file in paired_files:
    # Leer y procesar el archivo .hea
    with open(hea_file, 'r') as f:
        hea_content = f.read()

    # Leer el archivo .mat
    mat_content = loadmat(mat_file)

    # Extraer información específica según las necesidades
    record_name = hea_file.split("/")[-1].split(".")[0]
    duration_seconds = len(mat_content['val'][0]) / 250.0  # Duración en segundos
    alarm_type = None
    arrhythmia_type = None

    # Leer el tipo de alarma desde el archivo .hea
    if "True alarm" in hea_content:
        alarm_type = "True alarm"
    elif "False alarm" in hea_content:
        alarm_type = "False alarm"

    # Leer el tipo de arritmia desde el archivo .hea
    possible_comments = ["#Bradycardia", "#Ventricular_Tachycardia", "#Tachycardia", "#Ventricular_Flutter_Fib", "#Asystole"]
    arrhythmia_type = [comment[1:] for comment in possible_comments if comment in hea_content]

    # Agregar a la lista ecg_info
    ecg_info.append({"Record Name": record_name, "Duration (s)": duration_seconds,
                     "Alarm Type": alarm_type, "Arrhythmia Type": arrhythmia_type[0] if arrhythmia_type else None})

# Crear un DataFrame con la información de los ECG
info_df = pd.DataFrame(ecg_info)

# Imprimir la tabla de información
print("Tabla de Información de los Electrocardiogramas:")
print(info_df)


Tabla de Información de los Electrocardiogramas:
    Record Name  Duration (s)   Alarm Type          Arrhythmia Type
0         a103l         330.0  False alarm                 Asystole
1         a104s         300.0  False alarm                 Asystole
2         a105l         330.0  False alarm                 Asystole
3         a109l         330.0  False alarm                 Asystole
4         a123l         330.0  False alarm                 Asystole
..          ...           ...          ...                      ...
745       v843l         330.0  False alarm  Ventricular_Tachycardia
746       v844s         300.0   True alarm  Ventricular_Tachycardia
747       v845l         330.0  False alarm  Ventricular_Tachycardia
748       v846s         300.0  False alarm  Ventricular_Tachycardia
749       v848s         300.0  False alarm  Ventricular_Tachycardia

[750 rows x 4 columns]


Imprimir la tabla de frecuencias para el Tipo de Arritmia:

In [ ]:
# Generar la tabla de frecuencias
arrhythmia_table = info_df.groupby(["Alarm Type", "Arrhythmia Type"]).size().reset_index(name='Conteo')

# Agregar una fila para el total al final de la tabla
total_row = {"Alarm Type": "Total", "Arrhythmia Type": "", "Conteo": arrhythmia_table["Conteo"].sum()}
arrhythmia_table = pd.concat([arrhythmia_table, pd.DataFrame([total_row])], ignore_index=True)

# Imprimir la tabla de frecuencias
print("\nTabla de Frecuencias - Tipo de Arritmia con respecto al Tipo de Alarma:")
print(arrhythmia_table)


Tabla de Frecuencias - Tipo de Arritmia con respecto al Tipo de Alarma:
     Alarm Type          Arrhythmia Type  Conteo
0   False alarm                 Asystole     100
1   False alarm              Bradycardia      43
2   False alarm              Tachycardia       9
3   False alarm  Ventricular_Flutter_Fib      52
4   False alarm  Ventricular_Tachycardia     252
5    True alarm                 Asystole      22
6    True alarm              Bradycardia      46
7    True alarm              Tachycardia     131
8    True alarm  Ventricular_Flutter_Fib       6
9    True alarm  Ventricular_Tachycardia      89
10        Total                              750


Imprimir la tabla de frecuencias para el Tipo de Alarma:

In [ ]:
# Generar la tabla de frecuencias para el tipo de alarma
alarm_table = info_df.groupby("Alarm Type").size().reset_index(name='Conteo')

# Agregar una fila para el total al final de la tabla
total_row_alarm = {"Alarm Type": "Total", "Conteo": alarm_table["Conteo"].sum()}
alarm_table = pd.concat([alarm_table, pd.DataFrame([total_row_alarm])], ignore_index=True)

# Imprimir la tabla de frecuencias para el tipo de alarma
print("\nTabla de Frecuencias - Tipo de Alarma:")
print(alarm_table)



Tabla de Frecuencias - Tipo de Alarma:
    Alarm Type  Conteo
0  False alarm     456
1   True alarm     294
2        Total     750


Imprimir la tabla de frecuencias para Duración de los registro:

In [ ]:
# Contar la cantidad de datos para cada duración
duration_counts = info_df['Duration (s)'].value_counts().reset_index()
duration_counts.columns = ['Duration (s)', 'Cantidad de Datos']

# Imprimir la tabla de frecuencias para la duración
print("\nTabla de Frecuencias - Duración:")
print(duration_counts)



Tabla de Frecuencias - Duración:
   Duration (s)  Cantidad de Datos
0         330.0                375
1         300.0                375
